Imports

In [241]:
import pandas as pd
import os
import re

print('hi')

hi


Set directory to folder containing csv files and make an array of csv files

In [242]:
# os.chdir('./Downloads/export/EMLI_UPDATE_20230824/EMLI_UPDATE_20230824')
# os.chdir('../../../Downloads/export/EMLI_UPDATE_20230824/EMLI_UPDATE_20230824')

os.chdir('../EMLI_UPDATE_20230824/')

csv_files = [f for f in os.listdir() if f.endswith('.csv')]
# print(csv_files)
sorted(csv_files)

FileNotFoundError: [Errno 2] No such file or directory: '../EMLI_UPDATE_20230824/'

Set each csv file to a dataframe

In [ ]:
dfs = []

for csv in csv_files:
    df = pd.read_csv(csv)
    dfs.append(df)
    
print(dfs[0].columns)
print(dfs[1].columns)
print(dfs[2].columns)
print(dfs[3].columns)

NameError: name 'csv_files' is not defined

Join dataframes

In [ ]:
# merged_df_1 = pd.merge(dfs[0], dfs[3], on=['TITLE_NMBR', 'PRMNNT_PRCL_ID'])
# merged_df_2 = pd.merge(dfs[1], dfs[2], on=['TITLE_NMBR', 'LTB_DISTRICT_CD'])
# final_df = pd.merge(merged_df_1, merged_df_2, on=['TITLE_NMBR', 'LTB_DISTRICT_CD'])
# print(final_df)

merged_df_1 = pd.merge(dfs[0], dfs[1], on=['TITLE_NMBR', 'LTB_DISTRICT_CD'])
merged_df_2 = pd.merge(dfs[2], dfs[3], on='TITLE_NMBR')
final_df = pd.merge(merged_df_1, merged_df_2, on='TITLE_NMBR')

final_df


IndexError: list index out of range

Remove unecessary columns

In [ ]:
final_df = final_df.drop(columns=['NATURE_OF_XFER1', 'NATURE_OF_XFER2', 'TTL_ENTRD_DT', 'TTL_CNCL_DT', 'DCMNT_ACPTNC_DT', 'MRKT_VALUE_AMNT', 'REGISTRATION_DATE', 'CANCELLATION_DATE', 'REGDES', 'TX_ATHRTY_NM', 'LGDS', 'AIRSPACE_IND', 'STRATA_PLAN_NMBR', 'NMRTR_NMBR', 'DNMNTR_NMBR', 'TNNCY_TYP_IND', 'OCCPTN_DESC', 'TTL_RMRK_TEXT'])
final_df = final_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
# print(final_df)
final_df.columns

NameError: name 'final_df' is not defined

In [ ]:
little_df = final_df.query('`TITLE_NMBR` == "A13613" or  `TITLE_NMBR` == "AC206761" or `TITLE_NMBR` == "AD134401"')


In [526]:
output_series = final_df.groupby('TITLE_NMBR')['PRMNNT_PRCL_ID_x'].apply(list).reset_index(name='PIDs')


NameError: name 'final_df' is not defined

In [ ]:
output_series

,TITLE_NMBR,PIDs
0,A10055,[7753632]
1,A1030,[14174332]
2,A10826,"[9029214, 9029214]"
3,A11082,"[14219425, 14219425]"
4,A11113,[9317627]
...,...,...
49988,BB141289,[8704724]
49989,BB1414,[7818602]
49990,BB141426,"[4638875, 4638875]"
49991,BB141430,"[15161544, 15161544]"


In [ ]:
output_series['PIDs'] = output_series['PIDs'].tolist()
# type(output_series['PIDs'])

output_series['PIDs'] = output_series['PIDs'].apply(lambda x: str(sorted(list(set(x)))).replace("[", "").replace("]", "").replace(",", " | "))

# for pids in output_series['PIDs']:
#     index = (output_series['PIDs'].tolist()).index(pids)
#     pids = str(sorted(list(set(pids))))
#     pids = pids.replace("[", "").replace("]", "").replace(",", " | ")
#     output_series['PIDs'][index] = pids

output_series

,TITLE_NMBR,PIDs
0,A10055,7753632
1,A1030,14174332
2,A10826,9029214
3,A11082,14219425
4,A11113,9317627
...,...,...
49988,BB141289,8704724
49989,BB1414,7818602
49990,BB141426,4638875
49991,BB141430,15161544


In [ ]:
new_df = pd.merge(final_df, output_series, on='TITLE_NMBR')
new_df = new_df.drop(columns=['PRMNNT_PRCL_ID_x', 'PRMNNT_PRCL_ID_y', 'LTB_DISTRICT_CD_x', 'LTB_DISTRICT_CD_y','TTL_OWNRSHP_NMBR', 'PRCL_STTS_CD'])

new_df = new_df.loc[new_df.astype(str).drop_duplicates().index]

new_df.columns

Index(['TITLE_NMBR', 'CLIENT_GVN_NM', 'CLIENT_LST_NM_1', 'CLIENT_LST_NM_2',
       'INCRPRTN_NMBR', 'ADDRS_DESC_1', 'ADDRS_DESC_2', 'ADDRS_CITY',
       'ADDRS_PROV_CD', 'ADDRS_PROV_ST', 'ADDRS_CNTRY', 'ADDRS_PSTL_CD',
       'LTB_DISTRICT_CD', 'TTL_STTS_CD', 'FRM_TTL_NMBR', 'FRM_LT_DISTRICT_CD',
       'PIDs'],
      dtype='object')

In [ ]:
new_df.columns

Index(['TITLE_NMBR', 'CLIENT_GVN_NM', 'CLIENT_LST_NM_1', 'CLIENT_LST_NM_2',
       'INCRPRTN_NMBR', 'ADDRS_DESC_1', 'ADDRS_DESC_2', 'ADDRS_CITY',
       'ADDRS_PROV_CD', 'ADDRS_PROV_ST', 'ADDRS_CNTRY', 'ADDRS_PSTL_CD',
       'LTB_DISTRICT_CD', 'TTL_STTS_CD', 'FRM_TTL_NMBR', 'FRM_LT_DISTRICT_CD',
       'PIDs'],
      dtype='object')

In [ ]:
new_df.query('`TITLE_NMBR` == "A13613" or  `TITLE_NMBR` == "AC206761" or `TITLE_NMBR` == "AD134401"')

,TITLE_NMBR,CLIENT_GVN_NM,CLIENT_LST_NM_1,CLIENT_LST_NM_2,INCRPRTN_NMBR,ADDRS_DESC_1,ADDRS_DESC_2,ADDRS_CITY,ADDRS_PROV_CD,ADDRS_PROV_ST,ADDRS_CNTRY,ADDRS_PSTL_CD,LTB_DISTRICT_CD,TTL_STTS_CD,FRM_TTL_NMBR,FRM_LT_DISTRICT_CD,PIDs
14891,AC206761,KAZUTO,MIYAUCHI,NaN,,8333 AEGEAN CRESCENT,,VANCOUVER,BC,BC,CANADA,V5S 4H9,NW,R,M66550E,NW,19006608
20506,AD134401,CHRISTINE GLORIA,COLLINS,NaN,,24 - 8400 SHOOK ROAD,,MISSION,BC,BC,CANADA,V2V 5M2,NW,R,AD127369,NW,19006608 | 19006616
26813,A13613,NaN,HER MAJESTY THE QUEEN IN RIGHT OF CANADA,NaN,,C/O MINSTER OF INDIAN AFFAIRS AND NORTHERN DEV...,,OTTAWA,ON,,CANADA,,VI,R,84809W,VI,3955290 | 3955311 | 3955346


In [ ]:
import pandas as pd 
# Specify the columns you want to read by name
columns_to_read = ['Column1', 'Column3', 'Column5'] 

# # Read the CSV file while selecting only the specified columns
df = pd.read_csv('your_file.csv', usecols=columns_to_read)

KeyboardInterrupt: 

In [ ]:
# input_directory = './Downloads/export/EMLI_UPDATE_20230824/EMLI_UPDATE_20230824'
input_directory = ''
title_df = pd.read_csv(input_directory)

FileNotFoundError: [Errno 2] No such file or directory: '../Downloads/'

In [ ]:
input_directory = '/Users/emendelson/Downloads/export/EMLI_UPDATE_20230824/EMLI_UPDATE_20230824/'

title_df = pd.read_csv(input_directory+'1_title.csv')
parcel_df = pd.read_csv(input_directory+'2_parcel.csv')
title_parcel_df = pd.read_csv(input_directory+'3_titleparcel.csv')
title_owner_df = pd.read_csv(input_directory+'4_titleowner.csv')

In [ ]:
title_df.head()

,TITLE_NMBR,LTB_DISTRICT_CD,TTL_STTS_CD,FRM_TTL_NMBR,FRM_LT_DISTRICT_CD,NATURE_OF_XFER1,NATURE_OF_XFER2,TTL_ENTRD_DT,TTL_CNCL_DT,DCMNT_ACPTNC_DT,MRKT_VALUE_AMNT
0,AA100060E,NW,R,RD68051E,NW,FEE,,1987-06-03,NaN,1987-06-02,"$200,000.00"
1,AA100103E,NW,R,424032E,NW,SD,,1987-06-02,NaN,1987-06-02,N/A
2,AA100103E,NW,R,548278E,NW,SD,,1987-06-02,NaN,1987-06-02,N/A
3,AA100145E,NW,R,H56630E,NW,FEE,,1987-07-25,NaN,1987-06-02,"$45,000.00"
4,AA100176E,NW,R,Z9922E,NW,FEE,,1987-06-22,NaN,1987-06-02,


In [ ]:
print(title_owner_df.columns)
len(title_owner_df)

Index(['TITLE_NMBR', 'LTB_DISTRICT_CD', 'TTL_OWNRSHP_NMBR', 'NMRTR_NMBR',
       'DNMNTR_NMBR', 'TNNCY_TYP_IND', 'CLIENT_GVN_NM', 'CLIENT_LST_NM_1',
       'CLIENT_LST_NM_2', 'OCCPTN_DESC', 'INCRPRTN_NMBR', 'ADDRS_DESC_1',
       'ADDRS_DESC_2', 'ADDRS_CITY', 'ADDRS_PROV_CD', 'ADDRS_PROV_ST',
       'ADDRS_CNTRY', 'ADDRS_PSTL_CD', 'TTL_RMRK_TEXT'],
      dtype='object')


75680

In [ ]:
print(title_df.columns)
len(title_df)


Index(['TITLE_NMBR', 'LTB_DISTRICT_CD', 'TTL_STTS_CD', 'FRM_TTL_NMBR',
       'FRM_LT_DISTRICT_CD', 'NATURE_OF_XFER1', 'NATURE_OF_XFER2',
       'TTL_ENTRD_DT', 'TTL_CNCL_DT', 'DCMNT_ACPTNC_DT', 'MRKT_VALUE_AMNT'],
      dtype='object')


52443

In [ ]:
print(title_parcel_df.columns)
len(title_parcel_df)

Index(['TITLE_NMBR', 'LTB_DISTRICT_CD', 'PRMNNT_PRCL_ID'], dtype='object')


50020

In [ ]:
print(parcel_df.columns)
len(parcel_df)

Index(['PRMNNT_PRCL_ID', 'PRCL_STTS_CD', 'REGISTRATION_DATE',
       'CANCELLATION_DATE', 'REGDES', 'TX_ATHRTY_NM', 'LGDS', 'AIRSPACE_IND',
       'STRATA_PLAN_NMBR', 'TITLE_NMBR'],
      dtype='object')


50044

In [ ]:
merged_df_1 = pd.merge(title_owner_df, title_df, on="TITLE_NMBR")
print(merged_df_1.columns)
len(merged_df_1)

Index(['TITLE_NMBR', 'LTB_DISTRICT_CD_x', 'TTL_OWNRSHP_NMBR', 'NMRTR_NMBR',
       'DNMNTR_NMBR', 'TNNCY_TYP_IND', 'CLIENT_GVN_NM', 'CLIENT_LST_NM_1',
       'CLIENT_LST_NM_2', 'OCCPTN_DESC', 'INCRPRTN_NMBR', 'ADDRS_DESC_1',
       'ADDRS_DESC_2', 'ADDRS_CITY', 'ADDRS_PROV_CD', 'ADDRS_PROV_ST',
       'ADDRS_CNTRY', 'ADDRS_PSTL_CD', 'TTL_RMRK_TEXT', 'LTB_DISTRICT_CD_y',
       'TTL_STTS_CD', 'FRM_TTL_NMBR', 'FRM_LT_DISTRICT_CD', 'NATURE_OF_XFER1',
       'NATURE_OF_XFER2', 'TTL_ENTRD_DT', 'TTL_CNCL_DT', 'DCMNT_ACPTNC_DT',
       'MRKT_VALUE_AMNT'],
      dtype='object')


78490

In [ ]:
merged_df_2 = pd.merge(title_owner_df, title_df, on=["TITLE_NMBR", "LTB_DISTRICT_CD"])
print(merged_df_2.columns)
len(merged_df_2)

Index(['TITLE_NMBR', 'LTB_DISTRICT_CD', 'TTL_OWNRSHP_NMBR', 'NMRTR_NMBR',
       'DNMNTR_NMBR', 'TNNCY_TYP_IND', 'CLIENT_GVN_NM', 'CLIENT_LST_NM_1',
       'CLIENT_LST_NM_2', 'OCCPTN_DESC', 'INCRPRTN_NMBR', 'ADDRS_DESC_1',
       'ADDRS_DESC_2', 'ADDRS_CITY', 'ADDRS_PROV_CD', 'ADDRS_PROV_ST',
       'ADDRS_CNTRY', 'ADDRS_PSTL_CD', 'TTL_RMRK_TEXT', 'TTL_STTS_CD',
       'FRM_TTL_NMBR', 'FRM_LT_DISTRICT_CD', 'NATURE_OF_XFER1',
       'NATURE_OF_XFER2', 'TTL_ENTRD_DT', 'TTL_CNCL_DT', 'DCMNT_ACPTNC_DT',
       'MRKT_VALUE_AMNT'],
      dtype='object')


78476

In [ ]:
parcel_df = parcel_df.drop(
    columns=[
        "TITLE_NMBR",
    ]
)

In [ ]:
merged_df_3 = pd.merge(title_parcel_df, parcel_df, on="PRMNNT_PRCL_ID")
print(merged_df_3.columns)
len(merged_df_3)

Index(['TITLE_NMBR', 'LTB_DISTRICT_CD', 'PRMNNT_PRCL_ID', 'PRCL_STTS_CD',
       'REGISTRATION_DATE', 'CANCELLATION_DATE', 'REGDES', 'TX_ATHRTY_NM',
       'LGDS', 'AIRSPACE_IND', 'STRATA_PLAN_NMBR'],
      dtype='object')


52400

In [ ]:
merged_df_4 = pd.merge(merged_df_2, merged_df_3, on=["TITLE_NMBR", "LTB_DISTRICT_CD"])
print(merged_df_4.columns)
len(merged_df_4)

Index(['TITLE_NMBR', 'LTB_DISTRICT_CD', 'TTL_OWNRSHP_NMBR', 'NMRTR_NMBR',
       'DNMNTR_NMBR', 'TNNCY_TYP_IND', 'CLIENT_GVN_NM', 'CLIENT_LST_NM_1',
       'CLIENT_LST_NM_2', 'OCCPTN_DESC', 'INCRPRTN_NMBR', 'ADDRS_DESC_1',
       'ADDRS_DESC_2', 'ADDRS_CITY', 'ADDRS_PROV_CD', 'ADDRS_PROV_ST',
       'ADDRS_CNTRY', 'ADDRS_PSTL_CD', 'TTL_RMRK_TEXT', 'TTL_STTS_CD',
       'FRM_TTL_NMBR', 'FRM_LT_DISTRICT_CD', 'NATURE_OF_XFER1',
       'NATURE_OF_XFER2', 'TTL_ENTRD_DT', 'TTL_CNCL_DT', 'DCMNT_ACPTNC_DT',
       'MRKT_VALUE_AMNT', 'PRMNNT_PRCL_ID', 'PRCL_STTS_CD',
       'REGISTRATION_DATE', 'CANCELLATION_DATE', 'REGDES', 'TX_ATHRTY_NM',
       'LGDS', 'AIRSPACE_IND', 'STRATA_PLAN_NMBR'],
      dtype='object')


82074

In [ ]:
merged_df_4 = merged_df_4.drop(
    columns=[
        "NATURE_OF_XFER1",
        "NATURE_OF_XFER2",
        "TTL_ENTRD_DT",
        "TTL_CNCL_DT",
        "DCMNT_ACPTNC_DT",
        "MRKT_VALUE_AMNT",
        "REGISTRATION_DATE",
        "CANCELLATION_DATE",
        "REGDES",
        "TX_ATHRTY_NM",
        "LGDS",
        "AIRSPACE_IND",
        "STRATA_PLAN_NMBR",
        "NMRTR_NMBR",
        "DNMNTR_NMBR",
        "TNNCY_TYP_IND",
        "OCCPTN_DESC",
        "TTL_RMRK_TEXT",
        "PRCL_STTS_CD",
        "TTL_OWNRSHP_NMBR",
        # "CLIENT_LST_NM_2"  About to be dropped
    ]
)

print(merged_df_4.columns)

Index(['TITLE_NMBR', 'LTB_DISTRICT_CD', 'TTL_OWNRSHP_NMBR', 'CLIENT_GVN_NM',
       'CLIENT_LST_NM_1', 'CLIENT_LST_NM_2', 'INCRPRTN_NMBR', 'ADDRS_DESC_1',
       'ADDRS_DESC_2', 'ADDRS_CITY', 'ADDRS_PROV_CD', 'ADDRS_PROV_ST',
       'ADDRS_CNTRY', 'ADDRS_PSTL_CD', 'TTL_STTS_CD', 'FRM_TTL_NMBR',
       'FRM_LT_DISTRICT_CD', 'PRMNNT_PRCL_ID', 'PRCL_STTS_CD'],
      dtype='object')


In [ ]:
merged_df_4.head(100)

,TITLE_NMBR,LTB_DISTRICT_CD,TTL_OWNRSHP_NMBR,CLIENT_GVN_NM,CLIENT_LST_NM_1,CLIENT_LST_NM_2,INCRPRTN_NMBR,ADDRS_DESC_1,ADDRS_DESC_2,ADDRS_CITY,ADDRS_PROV_CD,ADDRS_PROV_ST,ADDRS_CNTRY,ADDRS_PSTL_CD,TTL_STTS_CD,FRM_TTL_NMBR,FRM_LT_DISTRICT_CD,PRMNNT_PRCL_ID,PRCL_STTS_CD
0,AA100060E,NW,1,ANN MARY,LIVESLEY,NaN,,10560 BLUNDELL ROAD ...,...,RICHMOND,BC,,CANADA,V6Y 1L1,R,RD68051E,NW,3631095,A
1,AA100060E,NW,1,WILLIAM JOHN,LIVESLEY,NaN,,10560 BLUNDELL ROAD ...,...,RICHMOND,BC,,CANADA,V6Y 1L1,R,RD68051E,NW,3631095,A
2,AA100103E,NW,1,NaN,THE HOLY TRINITY PARISH OF NEW WESTMINSTER B.C.,NaN,,514 CARNARVON STREET ...,...,NEW WESTMINSTER,BC,,CANADA,V3L 1C4,R,424032E,NW,8186430,A
3,AA100103E,NW,1,NaN,THE HOLY TRINITY PARISH OF NEW WESTMINSTER B.C.,NaN,,514 CARNARVON STREET ...,...,NEW WESTMINSTER,BC,,CANADA,V3L 1C4,R,548278E,NW,8186430,A
4,AA100145E,NW,1,ROY HENRY,LUMPKIN,NaN,,9169 PREST ROAD ...,...,CHILLIWACK,BC,,CANADA,V2P 6H3,R,H56630E,NW,7791178,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,AA106670E,NW,1,BRENDA CORRINNE RUBY,HENDERSON,NaN,,5205 CHAMBERLAYNE AVENUE ...,...,DELTA,BC,,CANADA,V4K 2J7,R,Y197002E,NW,2853264,A
96,AA106670E,NW,1,JOHN DOUGLAS,HENDERSON,NaN,,5205 CHAMBERLAYNE AVENUE ...,...,DELTA,BC,,CANADA,V4K 2J7,R,Y197002E,NW,2853264,A
97,AA106677E,NW,1,NaN,ALPI VENTURES LTD.,NaN,320587,3756 PANDORA STREET ...,...,BURNABY,BC,,CANADA,V5C 2A3,R,Z227984E,NW,5698758,A
98,AA106947E,NW,1,NaN,"DISTRICT OF MISSION,",NaN,,8645 STAVE LAKE STREET ...,...,MISSION,BC,,CANADA,V2V 4L9,R,Z70436E,NW,7765991,A


In [ ]:
print(str(merged_df_4["CLIENT_LST_NM_2"].unique()))

[nan
 'JR.                                                                        '
 '                                                                           '
 'MINISTRY OF FORESTS AND LANDS'
 '(MAPLE RIDGE - PITT MEADOWS)                                               '
 'REPRESENTED BY MINISTER OF TRANSPORTATION AND HIGHWAYS'
 'AKA, AHMED NIZAM                                                           '
 'JUNIOR                                                                     '
 '(BURNABY)                                                                  '
 'C/O MINISTRY OF ENVIRONMENT'
 'C/O MINISTRY OF ENVIRONMENT HARBOUR TOWERS' 'DISTRICT'
 'IN THE PROVINCE OF BRITISH COLUMBIA' 'REPRESENTED BY PARKS BRANCH'
 'AND DRAINAGE DISTRICT                                                      '
 'A STRATA CORPORATION UNDER THE STRATA PROPERTY ACT'
 'CARE OF THE MINISTER OF INDIAN AFFAIRS AND NORTHERN DEVELOPMENT'
 'C/O THE MINISTER OF INDIAN AFFAIRS AND NORTHERN DEVELOPMENT'
 'C/O MINI

In [ ]:
merged_df_4.isna().sum()

TITLE_NMBR                0
LTB_DISTRICT_CD           0
TTL_OWNRSHP_NMBR          0
CLIENT_GVN_NM         15039
CLIENT_LST_NM_1           1
CLIENT_LST_NM_2       81867
INCRPRTN_NMBR             0
ADDRS_DESC_1              0
ADDRS_DESC_2              0
ADDRS_CITY                0
ADDRS_PROV_CD             0
ADDRS_PROV_ST             0
ADDRS_CNTRY               0
ADDRS_PSTL_CD             0
TTL_STTS_CD               0
FRM_TTL_NMBR              0
FRM_LT_DISTRICT_CD        0
PRMNNT_PRCL_ID            0
PRCL_STTS_CD              0
dtype: int64

In [ ]:
merged_df_4.isnull().sum()

TITLE_NMBR                0
LTB_DISTRICT_CD           0
TTL_OWNRSHP_NMBR          0
CLIENT_GVN_NM         15064
CLIENT_LST_NM_1           1
CLIENT_LST_NM_2       81890
INCRPRTN_NMBR         76660
ADDRS_DESC_1           2466
ADDRS_DESC_2          80880
ADDRS_CITY             5596
ADDRS_PROV_CD          4360
ADDRS_PROV_ST         28007
ADDRS_CNTRY            3310
ADDRS_PSTL_CD          9996
TTL_STTS_CD               0
FRM_TTL_NMBR              0
FRM_LT_DISTRICT_CD        0
PRMNNT_PRCL_ID            0
PRCL_STTS_CD              0
dtype: int64

In [ ]:
print(str(merged_df_4["INCRPRTN_NMBR"].unique()))

[None '320587' '327826' ... '775313' 'S-576' '702834']


In [ ]:
merged_df_4 = merged_df_4.map(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
condition = merged_df_4["TITLE_NMBR"] == "AA10141E   "
print(merged_df_4[condition]["ADDRS_PROV_CD"])

Series([], Name: ADDRS_PROV_CD, dtype: object)


In [ ]:
merged_df_4.replace("", None, inplace = True)

In [ ]:
merged_df_1 = pd.merge(title_owner_df, title_df, on="TITLE_NMBR")
merged_df_2 = pd.merge(title_parcel_df, parcel_df, on="TITLE_NMBR")
active_pin_df = pd.merge(merged_df_1, merged_df_2, on="TITLE_NMBR")

In [ ]:
active_pin_df

,TITLE_NMBR,LTB_DISTRICT_CD,TTL_STTS_CD,FRM_TTL_NMBR,FRM_LT_DISTRICT_CD,NATURE_OF_XFER1,NATURE_OF_XFER2,TTL_ENTRD_DT,TTL_CNCL_DT,DCMNT_ACPTNC_DT,...,OCCPTN_DESC,INCRPRTN_NMBR,ADDRS_DESC_1,ADDRS_DESC_2,ADDRS_CITY,ADDRS_PROV_CD,ADDRS_PROV_ST,ADDRS_CNTRY,ADDRS_PSTL_CD,TTL_RMRK_TEXT
0,AA100060E,NW,R,RD68051E,NW,FEE,,1987-06-03,NaN,1987-06-02,...,HOMEMAKER ...,,10560 BLUNDELL ROAD ...,...,RICHMOND,BC,,CANADA,V6Y 1L1,NaN
1,AA100060E,NW,R,RD68051E,NW,FEE,,1987-06-03,NaN,1987-06-02,...,PHYSICIAN ...,,10560 BLUNDELL ROAD ...,...,RICHMOND,BC,,CANADA,V6Y 1L1,NaN
2,AA100103E,NW,R,424032E,NW,SD,,1987-06-02,NaN,1987-06-02,...,...,,514 CARNARVON STREET ...,...,NEW WESTMINSTER,BC,,CANADA,V3L 1C4,NaN
3,AA100103E,NW,R,548278E,NW,SD,,1987-06-02,NaN,1987-06-02,...,...,,514 CARNARVON STREET ...,...,NEW WESTMINSTER,BC,,CANADA,V3L 1C4,NaN
4,AA100145E,NW,R,H56630E,NW,FEE,,1987-07-25,NaN,1987-06-02,...,CEMENT MASON ...,,9169 PREST ROAD ...,...,CHILLIWACK,BC,,CANADA,V2P 6H3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78709,BB141426,NW,R,BF455634,NW,CA,,2007-12-10,NaN,2007-12-03,...,PEACE OFFICER,,4718 LONDON CRESCENT ...,...,DELTA,BC,BRITISH COLUMBIA,CANADA,V4K 4W8,NaN
78710,BB141430,VA,R,BV171087,VA,FEE,,2007-12-07,NaN,2007-12-03,...,VALUATION CONSULTANT,,1825 EAST 6TH AVENUE ...,...,VANCOUVER,BC,BRITISH COLUMBIA,CANADA,V5N 1P4,NaN
78711,BB141430,VA,R,BV171087,VA,FEE,,2007-12-07,NaN,2007-12-03,...,SALES REPRESENTATIVE,,1825 EAST 6TH AVENUE ...,...,VANCOUVER,BC,BRITISH COLUMBIA,CANADA,V5N 1P4,NaN
78712,BB141434,NW,R,BB568597,NW,249,FEE,2007-12-07,NaN,2007-12-03,...,I.T. PROFESSIONAL,,3080 EAST 21ST AVENUE ...,...,VANCOUVER,BC,BRITISH COLUMBIA,CANADA,V5M 2W7,NaN


In [546]:
import pandas as pd
import datetime
import numpy as np

In [547]:
input_directory = '/Users/emendelson/Downloads/export/EMLI_UPDATE_20230824/EMLI_UPDATE_20230824/'
title_df = pd.read_csv(
    input_directory + "1_title.csv",
    usecols=[
        "TITLE_NMBR",
        "LTB_DISTRICT_CD",
        "TTL_STTS_CD",
        "FRM_TTL_NMBR",
        "FRM_LT_DISTRICT_CD",
    ],
    dtype={
        "TITLE_NMBR": str,
        "LTB_DISTRICT_CD": str, 
        "TTL_STTS_CD": str,
        "FRM_TTL_NMBR": str,
        "FRM_LT_DISTRICT_CD": str,
    }
).map(lambda x: x.strip() if isinstance(x, str) else x).replace("", None).replace(np.nan, None)

title_df

,TITLE_NMBR,LTB_DISTRICT_CD,TTL_STTS_CD,FRM_TTL_NMBR,FRM_LT_DISTRICT_CD
0,AA100060E,NW,R,RD68051E,NW
1,AA100103E,NW,R,424032E,NW
2,AA100103E,NW,R,548278E,NW
3,AA100145E,NW,R,H56630E,NW
4,AA100176E,NW,R,Z9922E,NW
...,...,...,...,...,...
52438,BB141289,VA,R,BP154368,VA
52439,BB1414,PG,R,PN31937,PG
52440,BB141426,NW,R,BF455634,NW
52441,BB141430,VA,R,BV171087,VA


In [548]:
parcel_df = pd.read_csv(
    input_directory + "2_parcel.csv", 
    usecols=["PRMNNT_PRCL_ID", "PRCL_STTS_CD"],
    # usecols= lambda x: x.lower() in ["prmnnt_prcl_id"],
    dtype={
       "PRMNNT_PRCL_ID": int
    }
).map(lambda x: x.strip() if isinstance(x, str) else x).replace("", None).replace(np.nan, None)
print(parcel_df)

       PRMNNT_PRCL_ID PRCL_STTS_CD
0               48445            A
1               64661            A
2               67482            A
3               74993            A
4               84867            A
...               ...          ...
50039        28760280            A
50040        29209501            A
50041        29209994            A
50042        29295254            A
50043        29334268            A

[50044 rows x 2 columns]


In [549]:
title_parcel_df = pd.read_csv(
    input_directory + "3_titleparcel.csv",
    usecols=["TITLE_NMBR", "LTB_DISTRICT_CD", "PRMNNT_PRCL_ID"],
    dtype={
       "TITLE_NMBR": str, 
       "LTB_DISTRICT_CD": str, 
       "PRMNNT_PRCL_ID": int
    }
).map(lambda x: x.strip() if isinstance(x, str) else x).replace("", None).replace(np.nan, None)
title_parcel_df

,TITLE_NMBR,LTB_DISTRICT_CD,PRMNNT_PRCL_ID
0,AA100060E,NW,3631095
1,AA100103E,NW,8186430
2,AA100145E,NW,7791178
3,AA100176E,NW,3172953
4,AA100229E,NW,2757257
...,...,...,...
50015,BB141289,VA,8704724
50016,BB1414,PG,7818602
50017,BB141426,NW,4638875
50018,BB141430,VA,15161544


In [612]:
title_owner_df = pd.read_csv(
    input_directory + "4_titleowner.csv",
    usecols=[
        "TITLE_NMBR",
        "LTB_DISTRICT_CD",
        "CLIENT_GVN_NM",
        "CLIENT_LST_NM_1",
        "CLIENT_LST_NM_2",
        "INCRPRTN_NMBR",
        "ADDRS_DESC_1",
        "ADDRS_DESC_2",
        "ADDRS_CITY",
        "ADDRS_PROV_CD",
        "ADDRS_PROV_ST",
        "ADDRS_CNTRY",
        "ADDRS_PSTL_CD",
    ],
    dtype={
       "TITLE_NMBR": str,
        "LTB_DISTRICT_CD": str,
        "CLIENT_GVN_NM": str,
        "CLIENT_LST_NM_1": str,
        "CLIENT_LST_NM_1": str,
        "INCRPRTN_NMBR": str,
        "ADDRS_DESC_1": str,
        "ADDRS_DESC_2": str,
        "ADDRS_CITY": str,
        "ADDRS_PROV_CD": str,
        "ADDRS_PROV_ST": str,
        "ADDRS_CNTRY": str,
        "ADDRS_PSTL_CD": str,
    }
).map(lambda x: x.strip() if isinstance(x, str) else x).replace("", None).replace(np.nan, None)
title_owner_df

,TITLE_NMBR,LTB_DISTRICT_CD,CLIENT_GVN_NM,CLIENT_LST_NM_1,CLIENT_LST_NM_2,INCRPRTN_NMBR,ADDRS_DESC_1,ADDRS_DESC_2,ADDRS_CITY,ADDRS_PROV_CD,ADDRS_PROV_ST,ADDRS_CNTRY,ADDRS_PSTL_CD
0,AA100060E,NW,ANN MARY,LIVESLEY,None,None,10560 BLUNDELL ROAD,None,RICHMOND,BC,None,CANADA,V6Y 1L1
1,AA100060E,NW,WILLIAM JOHN,LIVESLEY,None,None,10560 BLUNDELL ROAD,None,RICHMOND,BC,None,CANADA,V6Y 1L1
2,AA100103E,NW,None,THE HOLY TRINITY PARISH OF NEW WESTMINSTER B.C.,None,None,514 CARNARVON STREET,None,NEW WESTMINSTER,BC,None,CANADA,V3L 1C4
3,AA100145E,NW,ROY HENRY,LUMPKIN,None,None,9169 PREST ROAD,None,CHILLIWACK,BC,None,CANADA,V2P 6H3
4,AA100176E,NW,BERNARD LEONARD,SLONOPAS,None,None,11964 CARRIAGE PLACE,None,DELTA,BC,None,CANADA,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75675,BB141426,NW,WAVERLEY ERVIN,STEINWAND,None,None,4718 LONDON CRESCENT,None,DELTA,BC,BRITISH COLUMBIA,CANADA,V4K 4W8
75676,BB141430,VA,SAMANTHA CHARLOTTE,SAWYER,None,None,1825 EAST 6TH AVENUE,None,VANCOUVER,BC,BRITISH COLUMBIA,CANADA,V5N 1P4
75677,BB141430,VA,TORBIN PAUL ARENDOLF,HOLST,None,None,1825 EAST 6TH AVENUE,None,VANCOUVER,BC,BRITISH COLUMBIA,CANADA,V5N 1P4
75678,BB141434,NW,DAVID HOAK YEW,CHO,None,None,3080 EAST 21ST AVENUE,None,VANCOUVER,BC,BRITISH COLUMBIA,CANADA,V5M 2W7


In [613]:
title_titleowner_df = pd.merge(
    title_owner_df, title_df, on=["TITLE_NMBR", "LTB_DISTRICT_CD"]
)
print(title_titleowner_df.count())
title_titleowner_df.head()


TITLE_NMBR            78476
LTB_DISTRICT_CD       78476
CLIENT_GVN_NM         63635
CLIENT_LST_NM_1       78475
CLIENT_LST_NM_2         179
INCRPRTN_NMBR          5236
ADDRS_DESC_1          76016
ADDRS_DESC_2           1177
ADDRS_CITY            72895
ADDRS_PROV_CD         74182
ADDRS_PROV_ST         50838
ADDRS_CNTRY           75182
ADDRS_PSTL_CD         68512
TTL_STTS_CD           78476
FRM_TTL_NMBR          78476
FRM_LT_DISTRICT_CD    78476
dtype: int64


,TITLE_NMBR,LTB_DISTRICT_CD,CLIENT_GVN_NM,CLIENT_LST_NM_1,CLIENT_LST_NM_2,INCRPRTN_NMBR,ADDRS_DESC_1,ADDRS_DESC_2,ADDRS_CITY,ADDRS_PROV_CD,ADDRS_PROV_ST,ADDRS_CNTRY,ADDRS_PSTL_CD,TTL_STTS_CD,FRM_TTL_NMBR,FRM_LT_DISTRICT_CD
0,AA100060E,NW,ANN MARY,LIVESLEY,None,None,10560 BLUNDELL ROAD,None,RICHMOND,BC,None,CANADA,V6Y 1L1,R,RD68051E,NW
1,AA100060E,NW,WILLIAM JOHN,LIVESLEY,None,None,10560 BLUNDELL ROAD,None,RICHMOND,BC,None,CANADA,V6Y 1L1,R,RD68051E,NW
2,AA100103E,NW,None,THE HOLY TRINITY PARISH OF NEW WESTMINSTER B.C.,None,None,514 CARNARVON STREET,None,NEW WESTMINSTER,BC,None,CANADA,V3L 1C4,R,424032E,NW
3,AA100103E,NW,None,THE HOLY TRINITY PARISH OF NEW WESTMINSTER B.C.,None,None,514 CARNARVON STREET,None,NEW WESTMINSTER,BC,None,CANADA,V3L 1C4,R,548278E,NW
4,AA100145E,NW,ROY HENRY,LUMPKIN,None,None,9169 PREST ROAD,None,CHILLIWACK,BC,None,CANADA,V2P 6H3,R,H56630E,NW


In [614]:
titleparcel_parcel_df = pd.merge(title_parcel_df, parcel_df, on="PRMNNT_PRCL_ID")
print(titleparcel_parcel_df.count())
titleparcel_parcel_df.head()

TITLE_NMBR         52400
LTB_DISTRICT_CD    52400
PRMNNT_PRCL_ID     52400
PRCL_STTS_CD       52400
dtype: int64


,TITLE_NMBR,LTB_DISTRICT_CD,PRMNNT_PRCL_ID,PRCL_STTS_CD
0,AA100060E,NW,3631095,A
1,AA100103E,NW,8186430,A
2,AA100145E,NW,7791178,A
3,AA100176E,NW,3172953,A
4,AA100229E,NW,2757257,A


In [615]:
active_pin_df = pd.merge(
    title_titleowner_df, titleparcel_parcel_df, on=["TITLE_NMBR", "LTB_DISTRICT_CD"]
)
print(active_pin_df.count())
active_pin_df.head()

TITLE_NMBR            82074
LTB_DISTRICT_CD       82074
CLIENT_GVN_NM         67010
CLIENT_LST_NM_1       82073
CLIENT_LST_NM_2         184
INCRPRTN_NMBR          5414
ADDRS_DESC_1          79608
ADDRS_DESC_2           1194
ADDRS_CITY            76478
ADDRS_PROV_CD         77714
ADDRS_PROV_ST         54067
ADDRS_CNTRY           78764
ADDRS_PSTL_CD         72078
TTL_STTS_CD           82074
FRM_TTL_NMBR          82074
FRM_LT_DISTRICT_CD    82074
PRMNNT_PRCL_ID        82074
PRCL_STTS_CD          82074
dtype: int64


,TITLE_NMBR,LTB_DISTRICT_CD,CLIENT_GVN_NM,CLIENT_LST_NM_1,CLIENT_LST_NM_2,INCRPRTN_NMBR,ADDRS_DESC_1,ADDRS_DESC_2,ADDRS_CITY,ADDRS_PROV_CD,ADDRS_PROV_ST,ADDRS_CNTRY,ADDRS_PSTL_CD,TTL_STTS_CD,FRM_TTL_NMBR,FRM_LT_DISTRICT_CD,PRMNNT_PRCL_ID,PRCL_STTS_CD
0,AA100060E,NW,ANN MARY,LIVESLEY,None,None,10560 BLUNDELL ROAD,None,RICHMOND,BC,None,CANADA,V6Y 1L1,R,RD68051E,NW,3631095,A
1,AA100060E,NW,WILLIAM JOHN,LIVESLEY,None,None,10560 BLUNDELL ROAD,None,RICHMOND,BC,None,CANADA,V6Y 1L1,R,RD68051E,NW,3631095,A
2,AA100103E,NW,None,THE HOLY TRINITY PARISH OF NEW WESTMINSTER B.C.,None,None,514 CARNARVON STREET,None,NEW WESTMINSTER,BC,None,CANADA,V3L 1C4,R,424032E,NW,8186430,A
3,AA100103E,NW,None,THE HOLY TRINITY PARISH OF NEW WESTMINSTER B.C.,None,None,514 CARNARVON STREET,None,NEW WESTMINSTER,BC,None,CANADA,V3L 1C4,R,548278E,NW,8186430,A
4,AA100145E,NW,ROY HENRY,LUMPKIN,None,None,9169 PREST ROAD,None,CHILLIWACK,BC,None,CANADA,V2P 6H3,R,H56630E,NW,7791178,A


In [616]:
active_pin_df = (
    active_pin_df.map(lambda x: x.strip() if isinstance(x, str) else x)
    .replace("", None)
    .replace(np.nan, None)
)
print(active_pin_df.count())
active_pin_df.head()

TITLE_NMBR            82074
LTB_DISTRICT_CD       82074
CLIENT_GVN_NM         67010
CLIENT_LST_NM_1       82073
CLIENT_LST_NM_2         184
INCRPRTN_NMBR          5414
ADDRS_DESC_1          79608
ADDRS_DESC_2           1194
ADDRS_CITY            76478
ADDRS_PROV_CD         77714
ADDRS_PROV_ST         54067
ADDRS_CNTRY           78764
ADDRS_PSTL_CD         72078
TTL_STTS_CD           82074
FRM_TTL_NMBR          82074
FRM_LT_DISTRICT_CD    82074
PRMNNT_PRCL_ID        82074
PRCL_STTS_CD          82074
dtype: int64


,TITLE_NMBR,LTB_DISTRICT_CD,CLIENT_GVN_NM,CLIENT_LST_NM_1,CLIENT_LST_NM_2,INCRPRTN_NMBR,ADDRS_DESC_1,ADDRS_DESC_2,ADDRS_CITY,ADDRS_PROV_CD,ADDRS_PROV_ST,ADDRS_CNTRY,ADDRS_PSTL_CD,TTL_STTS_CD,FRM_TTL_NMBR,FRM_LT_DISTRICT_CD,PRMNNT_PRCL_ID,PRCL_STTS_CD
0,AA100060E,NW,ANN MARY,LIVESLEY,None,None,10560 BLUNDELL ROAD,None,RICHMOND,BC,None,CANADA,V6Y 1L1,R,RD68051E,NW,3631095,A
1,AA100060E,NW,WILLIAM JOHN,LIVESLEY,None,None,10560 BLUNDELL ROAD,None,RICHMOND,BC,None,CANADA,V6Y 1L1,R,RD68051E,NW,3631095,A
2,AA100103E,NW,None,THE HOLY TRINITY PARISH OF NEW WESTMINSTER B.C.,None,None,514 CARNARVON STREET,None,NEW WESTMINSTER,BC,None,CANADA,V3L 1C4,R,424032E,NW,8186430,A
3,AA100103E,NW,None,THE HOLY TRINITY PARISH OF NEW WESTMINSTER B.C.,None,None,514 CARNARVON STREET,None,NEW WESTMINSTER,BC,None,CANADA,V3L 1C4,R,548278E,NW,8186430,A
4,AA100145E,NW,ROY HENRY,LUMPKIN,None,None,9169 PREST ROAD,None,CHILLIWACK,BC,None,CANADA,V2P 6H3,R,H56630E,NW,7791178,A


In [618]:
# Group by title number to get a list of pids associated with each title
titlenumber_pids_df = (
    # active_pin_df.drop()
    active_pin_df.groupby("TITLE_NMBR")["PRMNNT_PRCL_ID"]
    .apply(list)
    .reset_index(name="pids")
)
print(titlenumber_pids_df.count())
titlenumber_pids_df.head()

TITLE_NMBR    49993
pids          49993
dtype: int64


,TITLE_NMBR,pids
0,A10055,[7753632]
1,A1030,[14174332]
2,A10826,"[9029214, 9029214]"
3,A11082,"[14219425, 14219425]"
4,A11113,[9317627]


In [619]:
def pid_parser(pids):
    pids = (
        str(sorted(list(set(pids))))
        .replace("[", "")
        .replace("]", "")
        .replace(",", " |")
    )
    return pids

# For each title number in grouped_df, put list of pids in proper sorted string format
titlenumber_pids_df["pids"] = titlenumber_pids_df["pids"].apply(pid_parser)

titlenumber_pids_df.head()

,TITLE_NMBR,pids
0,A10055,7753632
1,A1030,14174332
2,A10826,9029214
3,A11082,14219425
4,A11113,9317627


In [620]:
# Merge dataframes to add in PIDs column and drop duplicate rows
active_pin_df = pd.merge(active_pin_df, titlenumber_pids_df, on="TITLE_NMBR").drop(
    columns=["PRMNNT_PRCL_ID"]
)
active_pin_df = active_pin_df.loc[active_pin_df.astype(str).drop_duplicates().index]
print(active_pin_df.count())
active_pin_df.head()

TITLE_NMBR            78281
LTB_DISTRICT_CD       78281
CLIENT_GVN_NM         63463
CLIENT_LST_NM_1       78280
CLIENT_LST_NM_2         179
INCRPRTN_NMBR          5229
ADDRS_DESC_1          75829
ADDRS_DESC_2           1174
ADDRS_CITY            72718
ADDRS_PROV_CD         74007
ADDRS_PROV_ST         50669
ADDRS_CNTRY           74996
ADDRS_PSTL_CD         68347
TTL_STTS_CD           78281
FRM_TTL_NMBR          78281
FRM_LT_DISTRICT_CD    78281
PRCL_STTS_CD          78281
pids                  78281
dtype: int64


,TITLE_NMBR,LTB_DISTRICT_CD,CLIENT_GVN_NM,CLIENT_LST_NM_1,CLIENT_LST_NM_2,INCRPRTN_NMBR,ADDRS_DESC_1,ADDRS_DESC_2,ADDRS_CITY,ADDRS_PROV_CD,ADDRS_PROV_ST,ADDRS_CNTRY,ADDRS_PSTL_CD,TTL_STTS_CD,FRM_TTL_NMBR,FRM_LT_DISTRICT_CD,PRCL_STTS_CD,pids
0,AA100060E,NW,ANN MARY,LIVESLEY,None,None,10560 BLUNDELL ROAD,None,RICHMOND,BC,None,CANADA,V6Y 1L1,R,RD68051E,NW,A,3631095
1,AA100060E,NW,WILLIAM JOHN,LIVESLEY,None,None,10560 BLUNDELL ROAD,None,RICHMOND,BC,None,CANADA,V6Y 1L1,R,RD68051E,NW,A,3631095
2,AA100103E,NW,None,THE HOLY TRINITY PARISH OF NEW WESTMINSTER B.C.,None,None,514 CARNARVON STREET,None,NEW WESTMINSTER,BC,None,CANADA,V3L 1C4,R,424032E,NW,A,8186430
3,AA100103E,NW,None,THE HOLY TRINITY PARISH OF NEW WESTMINSTER B.C.,None,None,514 CARNARVON STREET,None,NEW WESTMINSTER,BC,None,CANADA,V3L 1C4,R,548278E,NW,A,8186430
4,AA100145E,NW,ROY HENRY,LUMPKIN,None,None,9169 PREST ROAD,None,CHILLIWACK,BC,None,CANADA,V2P 6H3,R,H56630E,NW,A,7791178


In [621]:
active_pin_df = active_pin_df.rename(columns={
    "TITLE_NMBR": "title_number",
    "LTB_DISTRICT_CD": "land_title_district",
    "TTL_STTS_CD": "title_status",
    "FRM_TTL_NMBR": "from_title_number",
    "FRM_LT_DISTRICT_CD": "from_land_title_district",
    "CLIENT_GVN_NM": "given_name",
    "CLIENT_LST_NM_1": "last_name_1",
    "CLIENT_LST_NM_2": "last_name_2",
    "INCRPRTN_NMBR": "incorporation_number",
    "ADDRS_DESC_1": "address_line_1",
    "ADDRS_DESC_2": "address_line_2",
    "ADDRS_CITY": "city",
    "ADDRS_PROV_CD": "province_abbreviation",
    "ADDRS_PROV_ST": "province_long",
    "ADDRS_CNTRY": "country",
    "ADDRS_PSTL_CD": "postal_code",
})

In [622]:
processed_dataframe = active_pin_df
# print(processed_dataframe.loc[processed_dataframe['addrs_city'] == 'ABBOPSFORD'])

# processed_dataframe['addrs_city'] = processed_dataframe['addrs_city'].replace(dict.fromkeys(['ABBOPSFORD','ABOSTFORD','ABBOTOSFORD','ABBOTSFORED', 'ABBOTSFORD, B.C.', 'ABOTSFORD', 'ABBOTFORD', 'ABBOSTFORD'], 'ABBOTSFORD'))
print(processed_dataframe.loc[processed_dataframe['city'] == 'ABBOTSFORD'])

processed_dataframe

      title_number land_title_district         given_name last_name_1  \
68       AA104771E                  NW     BRIAN CUTHBERT       ADAMS   
69       AA104771E                  NW         VELMA JEAN       ADAMS   
87       AA106417E                  NW              LAURA        MORE   
88       AA106417E                  NW    LAWRENCE GEORGE        MORE   
104      AA107064E                  NW            DARSHAN       MAHIL   
...            ...                 ...                ...         ...   
81970     BB140436                  NW            ROBBERT  BINNENDIJK   
82042     BB141025                  NW  DINIE JOKE MARIAN   BINNENDYK   
82043     BB141025                  NW                JAN   BINNENDYK   
82063     BB141237                  NW         DAN EDWARD      SEARCY   
82064     BB141237                  NW        DEBORAH ANN      SEARCY   

      last_name_2 incorporation_number            address_line_1  \
68           None                 None  #5 - 33917 MARS

,title_number,land_title_district,given_name,last_name_1,last_name_2,incorporation_number,address_line_1,address_line_2,city,province_abbreviation,province_long,country,postal_code,title_status,from_title_number,from_land_title_district,PRCL_STTS_CD,pids
0,AA100060E,NW,ANN MARY,LIVESLEY,None,None,10560 BLUNDELL ROAD,None,RICHMOND,BC,None,CANADA,V6Y 1L1,R,RD68051E,NW,A,3631095
1,AA100060E,NW,WILLIAM JOHN,LIVESLEY,None,None,10560 BLUNDELL ROAD,None,RICHMOND,BC,None,CANADA,V6Y 1L1,R,RD68051E,NW,A,3631095
2,AA100103E,NW,None,THE HOLY TRINITY PARISH OF NEW WESTMINSTER B.C.,None,None,514 CARNARVON STREET,None,NEW WESTMINSTER,BC,None,CANADA,V3L 1C4,R,424032E,NW,A,8186430
3,AA100103E,NW,None,THE HOLY TRINITY PARISH OF NEW WESTMINSTER B.C.,None,None,514 CARNARVON STREET,None,NEW WESTMINSTER,BC,None,CANADA,V3L 1C4,R,548278E,NW,A,8186430
4,AA100145E,NW,ROY HENRY,LUMPKIN,None,None,9169 PREST ROAD,None,CHILLIWACK,BC,None,CANADA,V2P 6H3,R,H56630E,NW,A,7791178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82069,BB141426,NW,WAVERLEY ERVIN,STEINWAND,None,None,4718 LONDON CRESCENT,None,DELTA,BC,BRITISH COLUMBIA,CANADA,V4K 4W8,R,BF455634,NW,A,4638875
82070,BB141430,VA,SAMANTHA CHARLOTTE,SAWYER,None,None,1825 EAST 6TH AVENUE,None,VANCOUVER,BC,BRITISH COLUMBIA,CANADA,V5N 1P4,R,BV171087,VA,A,15161544
82071,BB141430,VA,TORBIN PAUL ARENDOLF,HOLST,None,None,1825 EAST 6TH AVENUE,None,VANCOUVER,BC,BRITISH COLUMBIA,CANADA,V5N 1P4,R,BV171087,VA,A,15161544
82072,BB141434,NW,DAVID HOAK YEW,CHO,None,None,3080 EAST 21ST AVENUE,None,VANCOUVER,BC,BRITISH COLUMBIA,CANADA,V5M 2W7,R,BB568597,NW,A,27288587


In [623]:
import json
import pandas as pd

In [635]:
with open('../cleaning_rules.json', 'r') as rule_file:
    data_cleaning = json.load(rule_file)

In [636]:
# Apply cleaning rules to each column
for column, rule in data_cleaning['column_rules'].items():
    # Replace Values
    if 'replace_exact_values' in rule.keys():
        for replacement in rule['replace_exact_values']:
            processed_dataframe[column] = processed_dataframe[column].replace(rule['replace_exact_values'][replacement], replacement)
    
    # Remove Characters
    if "remove_characters" in rule.keys():
        for replacement in rule["remove_characters"]:
            print(replacement)
            active_pin_df[column] = active_pin_df[column].str.replace(replacement, '')

    # Trim after comma
    if 'trim_map' in rule.keys():
        processed_dataframe[column] = processed_dataframe[column].apply(lambda x: x.split(',')[0] if isinstance(x, str) else x)

    # To lowercase
    if 'to_uppercase' in rule.keys():
        processed_dataframe[column] = processed_dataframe[column].apply(lambda x: x.upper() if isinstance(x, str) else x)
    


.
,
-
/
#
.
,
-
/
#
.
,
-
/
#


In [634]:
print(processed_dataframe.loc[processed_dataframe['city'] == 'ABBOTSFORED']) 
print(processed_dataframe.loc[processed_dataframe['city'] == 'FORT ST. JAMES'])

Empty DataFrame
Columns: [title_number, land_title_district, given_name, last_name_1, last_name_2, incorporation_number, address_line_1, address_line_2, city, province_abbreviation, province_long, country, postal_code, title_status, from_title_number, from_land_title_district, PRCL_STTS_CD, pids]
Index: []
Empty DataFrame
Columns: [title_number, land_title_district, given_name, last_name_1, last_name_2, incorporation_number, address_line_1, address_line_2, city, province_abbreviation, province_long, country, postal_code, title_status, from_title_number, from_land_title_district, PRCL_STTS_CD, pids]
Index: []


In [565]:
import os

cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))

Files in '/Users/emendelson/Documents/bc-emli-pin-mgmt-etl/utils': ['parse.py.ipynb', '__pycache__', 'sftp_download.py', 'ltsa_parse.py']
